In [1]:
%load_ext lab_black
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import torch
from torchvision import datasets, transforms
import os
import shutil

from tqdm import tqdm

In [3]:
# shutil.rmtree("train/n02096437")

In [44]:
# Load the ImageNet train dataset
train_dir = "train/"

train_dataset = datasets.ImageFolder(train_dir, transform=None)

# Get the number of images in the train dataset
num_images = len(train_dataset)

print(f"Number of images in the train folder: {num_images}")

Number of images in the train folder: 236816


In [ ]:
import zipfile
import tarfile


def get_uncompressed_size(tar_file_path):
    total_uncompressed_size = 0

    # Open the tar file
    with tarfile.open(tar_file_path, "r") as tar_file:
        # Iterate over each file in the tar archive
        for tar_info in tqdm(tar_file.getmembers()):
            # Accumulate the uncompressed size
            if tar_info.isfile():  # Ensure it's a file, not a directory or link
                total_uncompressed_size += tar_info.size

    return total_uncompressed_size


# Path to the zip file
zip_file_path = (
    "/home/soroush1/projects/def-kohitij/soroush1/imagenet/ILSVRC2012_img_train.tar"
)

# Calculate the total uncompressed size
uncompressed_size = get_uncompressed_size(zip_file_path)

# Convert the size to a more readable format (e.g., GB)
uncompressed_size_gb = uncompressed_size / (1024**3)

print(f"Total uncompressed size: {uncompressed_size_gb:.2f} GB")

In [ ]:
def get_total_size_of_tar(tar_file_path):
    total_size = 0

    def process_tar(tar_file):
        nonlocal total_size
        for member in tar_file.getmembers():
            if member.isfile():
                total_size += member.size
            if member.isfile() and member.name.endswith(".tar"):
                with tar_file.extractfile(member) as nested_tar_file_obj:
                    nested_tar_file_path = os.path.join("/tmp", member.name)
                    with open(nested_tar_file_path, "wb") as f:
                        f.write(nested_tar_file_obj.read())
                    with tarfile.open(nested_tar_file_path, "r") as nested_tar_file:
                        process_tar(nested_tar_file)
                    os.remove(nested_tar_file_path)

    # Open the outer tar file
    with tarfile.open(tar_file_path, "r") as tar_file:
        process_tar(tar_file)

    return total_size


# Path to the tar file
tar_file_path = (
    "/home/soroush1/projects/def-kohitij/soroush1/imagenet/ILSVRC2012_img_train.tar"
)

# Calculate the total uncompressed size of all files in the tar archive
total_size = get_total_size_of_tar(tar_file_path)

# Convert the size to a more readable format (e.g., GB)
total_size_gb = total_size / (1024**3)

print(
    f"Total uncompressed size of all files in the tar archive: {total_size_gb:.2f} GB"
)

In [ ]:
def list_tar_contents(tar_file_path):
    # Open the tar file
    with tarfile.open(tar_file_path, "r") as tar_file:
        # List all the files in the tar archive
        tar_file.list()


# List the contents of the tar file
list_tar_contents(
    tar_file_path="/home/soroush1/projects/def-kohitij/soroush1/imagenet/ILSVRC2012_img_train.tar"
)

In [ ]:
def count_files_in_tar(tar_file_path):
    file_count = 0

    # Open the tar file
    with tarfile.open(tar_file_path, "r") as tar_file:
        # Iterate over each member in the tar archive
        for tar_info in tar_file.getmembers():
            # Increment the count if it's a file
            if tar_info.isfile():
                file_count += 1

    return file_count


# List the contents of the tar file
count_files_in_tar(
    tar_file_path="/home/soroush1/projects/def-kohitij/soroush1/imagenet/ILSVRC2012_img_train.tar"
)

In [35]:
!rm -r train/

In [ ]:
import tarfile
import os


def extract_tar_files(tar_file_path, extract_dir):
    # Create the extraction directory if it doesn't exist
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    # Open the outer tar file
    with tarfile.open(tar_file_path, "r") as tar_file:
        # Iterate over each member in the tar archive
        for i, member in tqdm(enumerate(tar_file.getmembers()), total=1000):
            if member.isfile() and member.name.endswith(".tar"):
                # Create a folder for the nested tar file
                folder_name = os.path.join(
                    extract_dir, os.path.splitext(member.name)[0]
                )

                if not os.path.exists(folder_name):
                    os.makedirs(folder_name, exist_ok=True)

                    # Extract the nested tar file to the folder
                    nested_tar_path = os.path.join(folder_name, member.name)
                    with tar_file.extractfile(member) as nested_tar_file_obj:
                        with open(nested_tar_path, "wb") as f:
                            f.write(nested_tar_file_obj.read())

                    # Extract the contents of the nested tar file
                    with tarfile.open(nested_tar_path, "r") as nested_tar_file:
                        nested_tar_file.extractall(folder_name)

                    # Remove the nested tar file after extraction
                    os.remove(nested_tar_path)


# Path to the main tar file
tar_file_path = (
    "/home/soroush1/projects/def-kohitij/soroush1/imagenet/ILSVRC2012_img_train.tar"
)

# Directory to extract the contents to
extract_dir = "train"

# Extract the tar files
extract_tar_files(tar_file_path, extract_dir)

In [47]:
import tarfile
import os
from joblib import Parallel, delayed
from tqdm import tqdm


def extract_nested_tar(nested_tar_info, tar_file_path, extract_dir):
    with tarfile.open(tar_file_path, "r") as tar_file:
        nested_tar_file_obj = tar_file.extractfile(nested_tar_info)
        if nested_tar_file_obj is None:
            return

        folder_name = os.path.join(
            extract_dir, os.path.splitext(nested_tar_info.name)[0]
        )
        os.makedirs(folder_name, exist_ok=True)

        nested_tar_path = os.path.join(folder_name, nested_tar_info.name)
        with open(nested_tar_path, "wb") as f:
            f.write(nested_tar_file_obj.read())

        with tarfile.open(nested_tar_path, "r") as nested_tar_file:
            nested_tar_file.extractall(folder_name)

        os.remove(nested_tar_path)


def extract_tar_files(tar_file_path, extract_dir, n_jobs=-1):
    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with tarfile.open(tar_file_path, "r") as tar_file:
        members = [
            member
            for member in tar_file.getmembers()
            if member.isfile()
            and member.name.endswith(".tar")
            and not os.path.exists(
                os.path.join(extract_dir, os.path.splitext(member.name)[0])
            )
        ]
        print(len(members))

    Parallel(n_jobs=n_jobs)(
        delayed(extract_nested_tar)(member, tar_file_path, extract_dir)
        for member in tqdm(members)
    )


# Path to the main tar file
tar_file_path = (
    "/home/soroush1/projects/def-kohitij/soroush1/imagenet/ILSVRC2012_img_train.tar"
)

# Directory to extract the contents to
extract_dir = "/home/soroush1/nearline/def-kohitij/soroush1/train_folder"

# Number of parallel jobs (use -1 to use all CPUs, or specify a number)
n_jobs = 30

# Extract the tar files
extract_tar_files(tar_file_path, extract_dir, n_jobs)

1000




  0%|          | 0/1000 [00:00<?, ?it/s]

  3%|▎         | 30/1000 [00:00<00:08, 117.22it/s]

OSError: [Errno 122] Disk quota exceeded: '/home/soroush1/nearline/def-kohitij/soroush1/train_folder/n01514859/n01514859_8763.JPEG'



  3%|▎         | 30/1000 [00:16<00:08, 117.22it/s]